In [1]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [2]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# # filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

# _filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_perf_rank_dict =  'perf_rank_dict'  # store symbols from performance rank results
filename_pickled_r_all_ranks =  'r_all_ranks'  # list of top 100 most common symbols from performance rank results
filename_pickled_df_a = 'df_OHLCV_clean'  # df adjusted OHLCV, dropped symbols with no vol and close
filename_pickled_df_c = 'df_close_clean'  # df close, dropped symbols with no vol and close

verbose = False  # True prints more output

#################
# look_back_days = -250 * 60  # subset df iloc days
look_back_days = -250 * 6  # subset df iloc days, 6 years of data
#################

In [3]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [4]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_adjOHLCV[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
print(f'symbols with no volume:\n{symbols_no_volume}')

symbols with no volume:
Index(['AMCR', 'AVT', 'BHF', 'BIIB', 'BKR', 'CCI', 'CHD', 'CNC', 'CNXC',
       'ESAB', 'FCFS', 'GEN', 'HUBB', 'IBKR', 'JJC', 'MGI', 'PEAK', 'PFG',
       'PKI', 'ROCC', 'SBNY', 'SHEL', 'SITC', 'SRPT', 'TA', 'UA', 'VC', 'VNT',
       'WH', 'WLY', 'XEL'],
      dtype='object')


In [5]:
df_dif = df_v - df_v.shift(periods=1)
rows, cols = np.where(df_dif == 0)
idx_same_volume = list(set(cols))
idx_same_volume.sort()
idx_same_volume
symbols_same_volume = df_v.columns[idx_same_volume]
print(f'symbols with same volume:\n{symbols_same_volume}')

symbols with same volume:
Index(['ACM', 'ACN', 'ADP', 'AGCO', 'AMCR', 'AMED', 'AMG', 'AMGN', 'ANSS',
       'ARW',
       ...
       'VAC', 'VICR', 'VNDA', 'VRSK', 'VSAT', 'WERN', 'WEX', 'WTS', 'WTW',
       'XEL'],
      dtype='object', length=220)


In [6]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
print(f'symbols with NaN close:\n{symbols_no_close}')

symbols with NaN close:
Index(['ABB', 'ABNB', 'AIRC', 'ALGM', 'BHF', 'BJ', 'BOH', 'BRBR', 'BTC-USD',
       'CARR', 'CDAY', 'CEG', 'CHK', 'CHX', 'CNXC', 'CR', 'CRWD', 'CTVA',
       'DCP', 'DDOG', 'DOW', 'DT', 'DTM', 'ESAB', 'ETH-USD', 'ETRN', 'FISV',
       'FOX', 'FOXA', 'FRC', 'FYBR', 'GEHC', 'GFS', 'GO', 'GXO', 'JJC', 'KD',
       'LCID', 'MGI', 'MP', 'MRNA', 'NARI', 'NVST', 'NVT', 'OGN', 'OTIS',
       'PDD', 'PGNY', 'PKI', 'RIVN', 'ROCC', 'SBNY', 'SGOV', 'SHC', 'SITM',
       'SWAV', 'TA', 'VICI', 'VNT', 'VSCO', 'WH', 'YETI', 'ZM', 'ZS'],
      dtype='object')


In [7]:
symbols_drop = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
print(f'combined symbols with no volume, same volume and no close, inculdes duplicate symbols: {len(symbols_drop)}')
symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
symbols_drop .sort()
df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
print(f'unique symbols dropped from df_a (adjOLHLV) and df_c (Close): {len(symbols_drop)}')

combined symbols with no volume, same volume and no close, inculdes duplicate symbols: 315
unique symbols dropped from df_a (adjOLHLV) and df_c (Close): 289


In [8]:
print(f'symbols with no volume:   {len(symbols_no_volume):>5,}')
print(f'symbols with same volume: {len(symbols_same_volume):>5,}')
print(f'symbols with no close:    {len(symbols_no_close):>5,}\n')
print(f'symbols total before drop:                                        {len(df_close.columns):>5,}')
print(f'unique symbols dropped from df OHLCVA (df_a) and df Close (df_c): {len(symbols_drop):>5,}\n')
print('                                          symbols     rows')
print(f'df adjOHLCV (df_a) after dropped symbols:   {len(df_a.columns)/5:>5,.0f}    {len(df_a):>5,}')
print(f'df Close (df_c) after dropped symbols:      {len(df_c.columns):>5,}    {len(df_c):>5,}')


symbols with no volume:      31
symbols with same volume:   220
symbols with no close:       64

symbols total before drop:                                        1,583
unique symbols dropped from df OHLCVA (df_a) and df Close (df_c):   289

                                          symbols     rows
df adjOHLCV (df_a) after dropped symbols:   1,294    1,500
df Close (df_c) after dropped symbols:      1,294    1,500


In [9]:
pickle_dump(df_a, path_data_dump, filename_pickled_df_a)
print(f'pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_a}')
pickle_dump(df_c, path_data_dump, filename_pickled_df_c)
print(f'pickled df Close after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_c}')

pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCV_clean
pickled df Close after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_close_clean


In [10]:
from myUtils import list_dump

f_symbols_df_close_clean = 'symbols_df_close_clean.csv'  # symbols text file
symbols_df_c = list(df_c)  # column names in df_c
list_dump(symbols_df_c, path_data_dump, f_symbols_df_close_clean)# df_c.columns.to_csv(f_symbols_df_close_clean)

In [11]:
df_a.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2023-06-23  117.760002  119.709999  117.580002  119.419998  2632600.0   
2023-06-26  118.959999  119.800003  117.339996  118.370003  1673400.0   
2023-06-27  118.470001  118.959999  116.730003  117.040001  2326100.0   
2023-06-28  116.699997  117.110001  115.709999  116.370003  2196800.0   
2023-06-29  116.160004  118.250000  116.070000  118.050003  1885350.0   

                   AA                                              ...  \
                 Open       High        Low      Close     Volume  ...   
Date                                                               ...   
2023-06-23  32.730000  33.299999  32.549999  32.840000  5471400.0  ...   
2023-06-26  32.779999  33.529999  32.779999  32.990002  3311300.0  ...   
2023-06-27  33.490002  34.525002  33.029999  34.430000  4130400.0  ...   
2023-06-28  34.000000  34.029999  33.389999  33.660000  3657200.0  ...   
2023-06-29  33.500000  33.959999  33.080002  33.759998  5363070.0  ...   

                 ZUMZ                                                   ZWS  \
                 Open       High        Low      Close    Volume       Open   
Date                                                                          
2023-06-23  16.270000  16.959999  16.080000  16.870001  777900.0  25.370001   
2023-06-26  16.840000  17.250000  16.840000  17.040001  402000.0  25.400000   
2023-06-27  17.080000  17.290001  16.799999  17.180000  526200.0  25.459999   
2023-06-28  17.139999  17.139999  16.430000  16.580000  287100.0  26.040001   
2023-06-29  16.650000  17.150000  16.650000  16.980000  270078.0  26.270000   

                                                        
                 High        Low      Close     Volume  
Date                                                    
2023-06-23  26.030001  25.370001  25.480000  8485000.0  
2023-06-26  25.790001  25.270000  25.370001  1078900.0  
2023-06-27  26.100000  25.340000  26.040001  1037900.0  
2023-06-28  26.170000  25.879999  26.030001   691800.0  
2023-06-29  26.985001  26.240000  26.920000  1162561.0  

[5 rows x 6470 columns]

In [12]:
df_c.tail()

,A,AA,AAL,AAP,AAPL,AB,ABBV,ABC,ABM,ABR,...,YY,ZBH,ZBRA,ZD,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2023-06-23,119.419998,32.840000,16.260000,65.669998,186.679993,31.770000,135.919998,187.669998,40.529999,13.65,...,29.180000,143.990005,264.329987,64.860001,46.959999,26.600000,25.270000,168.369995,16.870001,25.480000
2023-06-26,118.370003,32.990002,16.440001,67.290001,185.270004,31.940001,134.949997,187.759995,41.360001,13.94,...,29.549999,142.809998,273.109985,66.220001,48.189999,27.080000,25.549999,167.539993,17.040001,25.370001
2023-06-27,117.040001,34.430000,17.350000,68.320000,188.059998,31.780001,132.600006,188.779999,41.500000,14.30,...,30.799999,144.809998,282.809998,67.260002,49.980000,27.299999,25.799999,170.669998,17.180000,26.040001
2023-06-28,116.370003,33.660000,17.549999,68.080002,189.250000,31.480000,132.509995,189.720001,41.939999,14.39,...,30.610001,144.000000,283.489990,68.320000,51.389999,27.090000,25.400000,169.100006,16.580000,26.030001
2023-06-29,118.050003,33.759998,17.600000,68.230003,189.589996,31.490000,133.229996,190.520004,42.630001,14.59,...,30.209999,145.149994,288.679993,70.099998,49.009998,27.280001,25.219999,171.869995,16.980000,26.920000
